In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from pathlib import Path

data_dir = Path('../input/tabular-playground-series-sep-2021/')

df_train = pd.read_csv(
    data_dir / "train.csv",
    index_col='id',
    nrows=25000,  # comment this row to use the full dataset
)

FEATURES = df_train.columns[:-1]
TARGET = df_train.columns[-1]

df_train.head()

**Model**

Let's try out a simple XGBoost model. This algorithm can handle missing values, but you could try imputing them instead. We use XGBClassifier (instead of XGBRegressor, for instance), since this is a classification problem.

In [ ]:
from xgboost import XGBClassifier

X = df_train.loc[:, FEATURES]
y = df_train.loc[:, TARGET]

model = XGBClassifier(
    max_depth=3,
    subsample=0.5,
    colsample_bytree=0.5,
    n_jobs=-1,
    # Uncomment if you want to use GPU. Recommended for whole training set.
    #tree_method='gpu_hist',
    random_state=0,
)

**Evaluation**

The evaluation metric is AUC, which stands for "area under curve". Run the next code cell to evaluate the model.

In [ ]:
from sklearn.model_selection import cross_validate
import warnings 
warnings.filterwarnings('ignore')

def score(X, y, model, cv):
    scoring = ["roc_auc"]
    scores = cross_validate(
        model, X, y, scoring=scoring, cv=cv, return_train_score=True
    )
    scores = pd.DataFrame(scores).T
    return scores.assign(
        mean = lambda x: x.mean(axis=1),
        std = lambda x: x.std(axis=1),
    )

scores = score(X, y, model, cv=2)

display(scores)

In [ ]:
# Fit on full training set
model.fit(X, y)

X_test = pd.read_csv(data_dir / "test.csv", index_col='id')

# Make predictions
y_pred = pd.Series(
    model.predict(X_test),
    index=X_test.index,
    name=TARGET,
)

# Create submission file
y_pred.to_csv("submission.csv")

**This is a simpler format to the approachable tabular dataset**